In [ ]:
#Import all the libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go
import plotly.express as px
from scipy.signal import medfilt, butter, filtfilt
import pywt
from sklearn.model_selection import train_test_split
import scipy.signal
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
#import the dataset from tensorflow datasets
df = pd.read_csv('/content/Final Ecg data set.csv', header=None)
df.head()
df.shape
df.info()
df.columns
df[140].unique()
df[140].value_counts() # 1 is normal & 0 is abnormal ecg
df.dtypes
df.isna().sum()
df.isna().sum().sum()
#plot graphs of normal and abnormal ECG to visualise the trends
abnormal = df[df.loc[:,140] ==0][:10]
normal = df[df.loc[:,140] ==1][:10]
# Create the figure
fig = go.Figure()
#create a list to display only a single legend
leg  = [False] * abnormal.shape[0]
leg[0] = True

# Plot training and validation error
for i in range(abnormal.shape[0]):
    fig.add_trace(go.Scatter( x=np.arange(abnormal.shape[1]),y=abnormal.iloc[i,:],name='Abnormal ECG', mode='lines',  marker_color='rgba(255, 0, 0, 0.9)', showlegend= leg[i]))
for j in range(normal.shape[0]):
    fig.add_trace(go.Scatter( x=np.arange(normal.shape[1]),y=normal.iloc[j,:],name='Normal ECG',  mode='lines',  marker_color='rgba(0, 255, 0, 1)', showlegend= leg[j]))
fig.update_layout(xaxis_title="time", yaxis_title="Signal", title= {'text': 'Difference between different ECG', 'xanchor': 'center', 'yanchor': 'top', 'x':0.5} , bargap=0,)
fig.update_traces(opacity=0.5)
fig.show()
# split the data into labels and features
ecg_data = df.iloc[:,:-1]
labels = df.iloc[:,-1]
# Normalize the data between -1 and 1
scaler = MinMaxScaler(feature_range=(-1, 1))
ecg_data = scaler.fit_transform(ecg_data)
#filtering the raw signals
# Median filtering
ecg_medfilt = medfilt(ecg_data, kernel_size=3)
# Low-pass filtering
lowcut = 0.05
highcut = 20.0
nyquist = 0.5 * 360.0
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(4, [low, high], btype='band')
ecg_lowpass = filtfilt(b, a, ecg_data)
# Wavelet filtering
coeffs = pywt.wavedec(ecg_data, 'db4', level=1)
threshold = np.std(coeffs[-1]) * np.sqrt(2*np.log(len(ecg_data)))
coeffs[1:] = (pywt.threshold(i, value=threshold, mode='soft') for i in coeffs[1:])
ecg_wavelet = pywt.waverec(coeffs, 'db4')
# Plot original ECG signal
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(ecg_data.shape[0]), y=ecg_data[30], mode='lines', name='Original ECG signal'))
# Plot filtered ECG signals
fig.add_trace(go.Scatter(x=np.arange(ecg_medfilt.shape[0]), y=ecg_medfilt[30], mode='lines', name='Median filtered ECG signal'))
fig.add_trace(go.Scatter(x=np.arange(ecg_lowpass.shape[0]), y=ecg_lowpass[30], mode='lines', name='Low-pass filtered ECG signal'))
fig.add_trace(go.Scatter(x=np.arange(ecg_wavelet.shape[0]), y=ecg_wavelet[30], mode='lines', name='Wavelet filtered ECG signal'))
fig.show()
#pad the signal with zeroes
def pad_data(original_data,filtered_data):
  # Calculate the difference in length between the original data and filtered data
  diff = original_data.shape[1] - filtered_data.shape[1]
    # pad the shorter array with zeroes
  if diff > 0:
          # Create an array of zeros with the same shape as the original data
      padding = np.zeros((filtered_data.shape[0], original_data.shape[1]))
      # Concatenate the filtered data with the padding
      padded_data = np.concatenate((filtered_data, padding))
  elif diff < 0:
      padded_data = filtered_data[:,:-abs(diff)]
  elif diff == 0:
      padded_data = filtered_data
  return padded_data
def mse(original_data, filtered_data):
    filter_data = pad_data(original_data,filtered_data)
    return np.mean((original_data - filter_data) ** 2)
# Calculate MSE
mse_value_m = mse(ecg_data, ecg_medfilt)
mse_value_l = mse(ecg_data, ecg_lowpass)
mse_value_w = mse(ecg_data, ecg_wavelet)
print("MSE value of Median Filtering:", mse_value_m)
print("MSE value of Low-pass Filtering:", mse_value_l)
print("MSE value of Wavelet Filtering:", mse_value_w)
# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(ecg_wavelet, labels, test_size=0.2, random_state=42)
# Initializing an empty list to store the features
features = []
# Extracting features for each sample
for i in range(X_train.shape[0]):
    #Finding the R-peaks
    r_peaks = scipy.signal.find_peaks(X_train[i])[0]
    #Initialize lists to hold R-peak and T-peak amplitudes
    r_amplitudes = []
    t_amplitudes = []
    # Iterate through R-peak locations to find corresponding T-peak amplitudes
    for r_peak in r_peaks:
        # Find the index of the T-peak (minimum value) in the interval from R-peak to R-peak + 200 samples
        t_peak = np.argmin(X_train[i][r_peak:r_peak+200]) + r_peak
        #Append the R-peak amplitude and T-peak amplitude to the lists
        r_amplitudes.append(X_train[i][r_peak])
        t_amplitudes.append(X_train[i][t_peak])
    # extracting singular value metrics from the r_amplitudes
    std_r_amp = np.std(r_amplitudes)
    mean_r_amp = np.mean(r_amplitudes)
    median_r_amp = np.median(r_amplitudes)
    sum_r_amp = np.sum(r_amplitudes)
    # extracting singular value metrics from the t_amplitudes
    std_t_amp = np.std(t_amplitudes)
    mean_t_amp = np.mean(t_amplitudes)
    median_t_amp = np.median(t_amplitudes)
    sum_t_amp = np.sum(t_amplitudes)
    # Find the time between consecutive R-peaks
    rr_intervals = np.diff(r_peaks)
    # Calculate the time duration of the data collection
    time_duration = (len(X_train[i]) - 1) / 1000 # assuming data is in ms
    # Calculate the sampling rate
    sampling_rate = len(X_train[i]) / time_duration
    # Calculate heart rate
    duration = len(X_train[i]) / sampling_rate
    heart_rate = (len(r_peaks) / duration) * 60
    # QRS duration
    qrs_duration = []
    for j in range(len(r_peaks)):
        qrs_duration.append(r_peaks[j]-r_peaks[j-1])
    # extracting singular value metrics from the qrs_durations
    std_qrs = np.std(qrs_duration)
    mean_qrs = np.mean(qrs_duration)
    median_qrs = np.median(qrs_duration)
    sum_qrs = np.sum(qrs_duration)
    # Extracting the singular value metrics from the RR-interval
    std_rr = np.std(rr_intervals)
    mean_rr = np.mean(rr_intervals)
    median_rr = np.median(rr_intervals)
    sum_rr = np.sum(rr_intervals)
    # Extracting the overall standard deviation
    std = np.std(X_train[i])
    # Extracting the overall mean
    mean = np.mean(X_train[i])
    # Appending the features to the list
    features.append([mean, std, std_qrs, mean_qrs,median_qrs, sum_qrs, std_r_amp, mean_r_amp, median_r_amp, sum_r_amp, std_t_amp, mean_t_amp, median_t_amp, sum_t_amp, sum_rr, std_rr, mean_rr,median_rr, heart_rate])
# Converting the list to a numpy array
features = np.array(features)
# Initializing an empty list to store the features
X_test_fe = []
# Extracting features for each sample
for i in range(X_test.shape[0]):
    # Finding the R-peaks
    r_peaks = scipy.signal.find_peaks(X_test[i])[0]
    # Initialize lists to hold R-peak and T-peak amplitudes
    r_amplitudes = []
    t_amplitudes = []
    # Iterate through R-peak locations to find corresponding T-peak amplitudes
    for r_peak in r_peaks:
        # Find the index of the T-peak (minimum value) in the interval from R-peak to R-peak + 200 samples
        t_peak = np.argmin(X_test[i][r_peak:r_peak+200]) + r_peak
        # Append the R-peak amplitude and T-peak amplitude to the lists
        r_amplitudes.append(X_test[i][r_peak])
        t_amplitudes.append(X_test[i][t_peak])
    #extracting singular value metrics from the r_amplitudes
    std_r_amp = np.std(r_amplitudes)
    mean_r_amp = np.mean(r_amplitudes)
    median_r_amp = np.median(r_amplitudes)
    sum_r_amp = np.sum(r_amplitudes)
    #extracting singular value metrics from the t_amplitudes
    std_t_amp = np.std(t_amplitudes)
    mean_t_amp = np.mean(t_amplitudes)
    median_t_amp = np.median(t_amplitudes)
    sum_t_amp = np.sum(t_amplitudes)
    # Find the time between consecutive R-peaks
    rr_intervals = np.diff(r_peaks)
    # Calculate the time duration of the data collection
    time_duration = (len(X_test[i]) - 1) / 1000 # assuming data is in ms
    # Calculate the sampling rate
    sampling_rate = len(X_test[i]) / time_duration
    # Calculate heart rate
    duration = len(X_test[i]) / sampling_rate
    heart_rate = (len(r_peaks) / duration) * 60
    # QRS duration
    qrs_duration = []
    for j in range(len(r_peaks)):
        qrs_duration.append(r_peaks[j]-r_peaks[j-1])
    #extracting singular value metrics from the qrs_duartions
    std_qrs = np.std(qrs_duration)
    mean_qrs = np.mean(qrs_duration)
    median_qrs = np.median(qrs_duration)
    sum_qrs = np.sum(qrs_duration)
    # Extracting the standard deviation of the RR-interval
    std_rr = np.std(rr_intervals)
    mean_rr = np.mean(rr_intervals)
    median_rr = np.median(rr_intervals)
    sum_rr = np.sum(rr_intervals)
      # Extracting the standard deviation of the RR-interval
    std = np.std(X_test[i])
    # Extracting the mean of the RR-interval
    mean = np.mean(X_test[i])
    # Appending the features to the list
    X_test_fe.append([mean, std,  std_qrs, mean_qrs,median_qrs, sum_qrs, std_r_amp, mean_r_amp, median_r_amp, sum_r_amp, std_t_amp, mean_t_amp, median_t_amp, sum_t_amp, sum_rr, std_rr, mean_rr,median_rr,heart_rate])
# Converting the list to a numpy array
X_test_fe = np.array(X_test_fe)
# Define the number of features in the train dataframe
num_features = features.shape[1]
# Reshape the features data to be in the right shape for LSTM input
features = np.asarray(features).astype('float32')
features = features.reshape(features.shape[0], features.shape[1], 1)
X_test_fe = X_test_fe.reshape(X_test_fe.shape[0], X_test_fe.shape[1], 1)
# Define the model architecture
model = Sequential()
model.add(LSTM(64, input_shape=(features.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(features, y_train, validation_data=(X_test_fe, y_test), epochs=50, batch_size=32)
# Make predictions on the validation set
y_pred = model.predict(X_test_fe)
# Convert the predicted values to binary labels
y_pred = [1 if p>0.5 else 0 for p in y_pred]
X_test_fe = np.asarray(X_test_fe).astype('float32')
# calculate the accuracy
acc = accuracy_score(y_test, y_pred)
#calculate the AUC score
auc = round(roc_auc_score(y_test, y_pred),2)
#classification report provides all metrics e.g. precision, recall, etc.
all_met = classification_report(y_test, y_pred)
# Print the accuracy
print("Accuracy: ", acc*100, "%")
print(" n")
print("AUC:", auc)
print(" n")
print("Classification Report: n", all_met)
print(" n")
# Calculate the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat_df = pd.DataFrame(conf_mat, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
fig = px.imshow(conf_mat_df, text_auto= True, color_continuous_scale='Blues')
fig.update_xaxes(side='top', title_text='Predicted')
fig.update_yaxes(title_text='Actual')
fig.show()
# Plot training and validation error
fig = go.Figure()
fig.add_trace(go.Scatter( y=history.history['loss'], mode='lines', name='Training'))
fig.add_trace(go.Scatter( y=history.history['val_loss'], mode='lines', name='Validation'))
fig.update_layout(xaxis_title="Epoch", yaxis_title="Error", title= {'text': 'Model Error', 'xanchor': 'center', 'yanchor': 'top', 'x':0.5} , bargap=0)
fig.show()
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Classification Report
cr = classification_report(y_test, y_pred)
print("Classification Report:\n", cr)

# ROC AUC Score
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC AUC Score:", roc_auc)

# Plotting training and validation loss
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(history.history['loss'])), y=history.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(history.history['val_loss'])), y=history.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Training & Validation Loss Over Epochs', xaxis_title='Epochs', yaxis_title='Loss', legend_title='Type')
fig.show()

# Plotting ROC Curve
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
fig = px.area(x=fpr, y=tpr, title=f'ROC Curve (AUC = {roc_auc:.4f})', labels=dict(x='False Positive Rate', y='True Positive Rate'), width=700, height=500)
fig.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)
fig.show()
import requests
import numpy as np
import pandas as pd
import json
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Function to send data to Ubidots with enhanced error handling
def send_data_to_ubidots(token, device_label, data):
    url = f"https://industrial.api.ubidots.com/api/v1.6/devices/{device_label}/"
    headers = {
        "Content-Type": "application/json",
        "X-Auth-Token": token,
    }

    try:
        response = requests.post(url, json=data, headers=headers)

        if response.status_code == 201:
            print("Data successfully sent to Ubidots.")
        else:
            print(f"Failed to send data to Ubidots. Status code: {response.status_code}")
            print("Response content:")
            print(json.dumps(response.json(), indent=2))

            if response.status_code == 200:
                print("Unexpected status code 200. Check the request data and endpoint.")
            elif response.status_code == 404:
                print("Error 404: Device not found. Check your device label and token.")
            elif response.status_code == 401:
                print("Error 401: Unauthorized. Check your API token.")
            elif response.status_code == 400:
                print("Error 400: Bad Request. There might be an error in the request data.")
            elif response.status_code == 500:
                print("Error 500: Server error at Ubidots.")
            else:
                print("An unknown error occurred.")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while sending data: {e}")


# Load ECG data from a CSV file (update path as needed)
data = pd.read_csv('/content/Final Ecg data set.csv', header=None)

# Ensure the dataset has 141 columns (140 features + 1 target)
assert data.shape[1] == 141, "Dataset should have 141 columns: 140 features and 1 target."

# Separate the features and the target variable
features = data.iloc[:, 0:140]  # 140 features (timepoints)
target = data.iloc[:, 140]      # 1 target variable

# Normalize the features
scaler = MinMaxScaler(feature_range=(-1, 1))
features_normalized = scaler.fit_transform(features)

# Train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(features_normalized, target, test_size=0.2, random_state=42)

# Define the LSTM model
def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize and train the model
model = create_model((X_train.shape[1], 1))
model.fit(X_train.reshape(X_train.shape[0], -1, 1), y_train, epochs=10, batch_size=32)

# Test the model and get accuracy
y_pred = (model.predict(X_test.reshape(X_test.shape[0], -1, 1)) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)

# Output the test accuracy
print(f"Model test accuracy: {accuracy:.2f}")

# Ask the user for analysis choice: random or user input
choice = input("Select analysis type: 'a' for random or 'b' to input row number: ").strip().lower()

if choice == 'a':
    # Random row analysis
    random_row_index = np.random.randint(data.shape[0])
    selected_row = features_normalized[random_row_index]
    name = f"Row {random_row_index + 1}"

elif choice == 'b':
    # User-specified row analysis
    while True:
        try:
            user_row_index = int(input(f"Enter a row number for analysis (1-{data.shape[0]}): ")) - 1
            if 0 <= user_row_index < data.shape[0]:
                selected_row = features_normalized[user_row_index]
                name = f"Row {user_row_index + 1}"
                break
            else:
                print(f"Invalid input. Please enter a number between 1 and {data.shape[0]}.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

else:
    raise ValueError("Invalid choice. Please enter 'a' or 'b'.")

# Normalize and reshape for prediction
selected_row_normalized = selected_row.reshape(1, -1, 1)

# Make a prediction
prediction = (model.predict(selected_row_normalized) > 0.5).astype(int)
predicted_class = 'Normal' if prediction[0][0] == 1 else 'Abnormal'

# Output the result
print(f"The ECG reading from {name} is predicted to be {predicted_class}.")

# Ubidots token and device label (update with your actual token and label)
TOKEN = 'BBUS-U3fWaL9knmZQTEwsL8OQ7wi7yCBgBs'  # Replace with your Ubidots API token
DEVICE_LABEL = 'ecg-device'  # Replace with your Ubidots device label

# Prepare the data to send to Ubidots
image_url = 'https://www.mediafire.com/view/5xljm77v18vfi3o/Results.jpg/file'  # Replace with your desired image URL

# Create message with prediction result and row information
message = f"The ECG reading from {name} is predicted to be {predicted_class}."

# Create data to send to Ubidots
data_to_send = {
    "ecg-status": {"value": 1, "context": {"Image URL": image_url}},
    "ecg-message": {"value": 1, "context": {"Message": message}},
}

# Send data to Ubidots
send_data_to_ubidots(TOKEN, DEVICE_LABEL, data_to_send)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Columns: 141 entries, 0 to 140
dtypes: float64(140), int64(1)
memory usage: 5.4 MB


MSE value of Median Filtering: 0.017269125263192336
MSE value of Low-pass Filtering: 0.36731639045665315
MSE value of Wavelet Filtering: 0.0010817856141850134
Epoch 1/50
125/125 [==============================] - 10s 35ms/step - loss: 0.4389 - accuracy: 0.7961 - val_loss: 0.2690 - val_accuracy: 0.9090
Epoch 2/50
125/125 [==============================] - 3s 26ms/step - loss: 0.2669 - accuracy: 0.9112 - val_loss: 0.2674 - val_accuracy: 0.9080
Epoch 3/50
125/125 [==============================] - 3s 24ms/step - loss: 0.2626 - accuracy: 0.9107 - val_loss: 0.2654 - val_accuracy: 0.9080
Epoch 4/50
125/125 [==============================] - 4s 33ms/step - loss: 0.2634 - accuracy: 0.9122 - val_loss: 0.2578 - val_accuracy: 0.9140
Epoch 5/50
125/125 [==============================] - 3s 27ms/step - loss: 0.2577 - accuracy: 0.9127 - val_loss: 0.2517 - val_accuracy: 0.9170
Epoch 6/50
125/125 [==============================] - 3s 24ms/step - loss: 0.2544 - accuracy: 0.9167 - val_loss: 0.2595 - val

Confusion Matrix:
 [[386  23]
 [ 47 544]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.94      0.92       409
           1       0.96      0.92      0.94       591

    accuracy                           0.93      1000
   macro avg       0.93      0.93      0.93      1000
weighted avg       0.93      0.93      0.93      1000

ROC AUC Score: 0.9321195272196229


Epoch 1/10
125/125 [==============================] - 11s 63ms/step - loss: 0.2540 - accuracy: 0.9367
Epoch 2/10
125/125 [==============================] - 9s 68ms/step - loss: 0.0775 - accuracy: 0.9735
Epoch 3/10
125/125 [==============================] - 8s 64ms/step - loss: 0.0596 - accuracy: 0.9810
Epoch 4/10
125/125 [==============================] - 8s 63ms/step - loss: 0.0520 - accuracy: 0.9845
Epoch 5/10
125/125 [==============================] - 9s 68ms/step - loss: 0.0493 - accuracy: 0.9852
Epoch 6/10
125/125 [==============================] - 7s 58ms/step - loss: 0.0505 - accuracy: 0.9840
Epoch 7/10
125/125 [==============================] - 9s 69ms/step - loss: 0.0481 - accuracy: 0.9850
Epoch 8/10
125/125 [==============================] - 8s 64ms/step - loss: 0.0466 - accuracy: 0.9860
Epoch 9/10
125/125 [==============================] - 8s 62ms/step - loss: 0.0478 - accuracy: 0.9857
Epoch 10/10
32/32 [==============================] - 1s 20ms/step
Model test accuracy: 0.9